In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [7]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs['accuracy']>DESIRED_ACCURACY):
      print('\nReached 99.9% accuracy so cancelling training!')
      self.model.stop_training = True
  # Your Code

callbacks = myCallback()

--2020-07-12 20:08:20--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.26.128, 172.217.204.128, 172.217.203.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.26.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2020-07-12 20:08:20 (105 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [8]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid')
    ])

from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [9]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    '/tmp/h-or-s',
    target_size=(150,150),
    class_mode='binary'
    )

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [10]:
# This code block should call model.fit and train for
# a number of epochs. 
history = model.fit(
    train_generator,
    epochs=20,
    callbacks=[callbacks])
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/20
3/3 [==============================] - 1s 307ms/step - loss: 4.0776 - accuracy: 0.3750
Epoch 2/20
3/3 [==============================] - 1s 392ms/step - loss: 0.7062 - accuracy: 0.4500
Epoch 3/20
3/3 [==============================] - 1s 390ms/step - loss: 0.6709 - accuracy: 0.6750
Epoch 4/20
3/3 [==============================] - 1s 303ms/step - loss: 0.6427 - accuracy: 0.5000
Epoch 5/20
3/3 [==============================] - 1s 398ms/step - loss: 0.8706 - accuracy: 0.5500
Epoch 6/20
3/3 [==============================] - 1s 389ms/step - loss: 0.6264 - accuracy: 0.8250
Epoch 7/20
3/3 [==============================] - 1s 292ms/step - loss: 0.6320 - accuracy: 0.5250
Epoch 8/20
3/3 [==============================] - 1s 290ms/step - loss: 0.3756 - accuracy: 0.9750
Epoch 9/20
3/3 [==============================] - 1s 299ms/step - loss: 0.2704 - accuracy: 0.9750
Epoch 10/20
3/3 [==============================] - 1s 390ms/step - loss: 0.2140 - accuracy: 0.8875
Epoch 11/20
3/3 [==